In [62]:
import re
import jieba
import jieba.analyse
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split


def csv_output(batch, file_path, classification, description):
    data = {
        '分類': classification,
        '劇情介紹': description,
    }
    classification_info = pd.DataFrame(data,index=[0]) #建立DataFrame物件
    if batch==0: #如果為第一次寫入
        classification_info.to_csv(file_path, encoding="utf-8", index=0) #寫入csv檔(含title)
    else:
        classification_info.to_csv(file_path, encoding="utf-8", mode='a', header=False, index=0) #寫入csv檔
        
        
def remain_language(string): #留下文字
    result = re.sub('\W+', ' ', string).replace("_",  '')
    result=result.replace(" ","")
    return result
        
    
def class_JiebaDes_output(file_path): #把 類別 跟 劇情介紹的分詞 寫入另一個檔案
    csv_content = pd.read_csv('../HW2/movie_info.csv') #讀取hw2的csv檔
    status=0
    jieba_words=[]

    for i in range(0, len(csv_content["名稱"])):
        if csv_content["分類"][i] != "[]":
            classification_first=re.findall('\[(\'\D+\')\]',csv_content["分類"][i]) #第一個類別
            classification_first=classification_first[0].split(",")[0].strip("'")

            jieba_words=jieba.lcut(remain_language(csv_content["劇情介紹"][i]))

            word_string=""
            for word in jieba_words:
                if word_string=="":
                    word_string=word
                else:
                    word_string=word_string+" "+word

            csv_output(status, file_path, classification_first, word_string)
            status=1
        


In [65]:

if __name__=="__main__":
    output_csv_path="class_description.csv"
    #class_JiebaDes_output(output_csv_path) 
    
    csv_content = pd.read_csv(output_csv_path)
    csv_content['class_id'] = csv_content['分類'].factorize()[0] #產生類別對應的id
    class_id_cate = csv_content[['分類', 'class_id']].drop_duplicates() #整理出總共有幾種類別

    class_to_id = dict(class_id_cate.values) #class_to_id[class name]=id
    id_to_class = dict(class_id_cate[['class_id', '分類']].values) #id_to_class[id]=class name
    
    tf_idf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 2), stop_words='english') 
    features = tf_idf.fit_transform(df.劇情介紹).toarray() 
    labels = csv_content.class_id 
    features.shape

    n = 30          
    svd = TruncatedSVD(n)               
    Features = svd.fit_transform(features)
    
    X_train, X_test, y_train, y_test = train_test_split(Features, df['分類'], test_size=500, random_state = 100)
    clf=KNeighborsClassifier(n_neighbors=31, p=2, weights="distance", algorithm='brute')
    clf.fit(X_train, y_train)
    
    print("正確率\n", clf.score(X_test, y_test)*100,"%")
    print("\n正確數據\t預測數據\n")
    for i in range(0,len(y_test)):
        print(y_test[i], "\t", clf.predict(X_test[i]))
    
    

正確率
 60.0 %

正確數據	預測數據



KeyError: 0